# Programming and Data Analysis

> Project: Taiwan Election 2022

Kuo, Yao-Jen <yaojenkuo@ntu.edu.tw> from [DATAINPOINT](https://www.datainpoint.com/)

In [1]:
# Importing libraries/functions the project needs
import os
import re
import numpy as np
import pandas as pd

## Importing Data

## Data source

歷屆公職選舉資料：<https://db.cec.gov.tw/ElecTable/Election>

## The original data format is Excel workbook

We can use `pd.read_excel` for importing.

## Importing the data of `縣表3-1-63000(臺北市)-111年臺北市縣市長選舉候選人在各投開票所得票數一覽表.xls`

Check the list of worksheets before importing.

In [2]:
file_name = "data/縣表3-1-63000(臺北市)-111年臺北市縣市長選舉候選人在各投開票所得票數一覽表.xls"
df = pd.read_excel(file_name)
df.head()

,111年臺北市市長選舉候選人在各投開票所得票數一覽表,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,行政區別,村里別,投開票所別,各候選人得票情形,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,A\n有效票數\n\nA=1+2+...N,B\n無效票數\n\n,C\n投票數\n\nC=A+B,D\n已領未投\n票 數\nD=E-C,E\n發出票數\n\nE=C+D,F\n用餘票數\n\n,G\n選舉人數\n\nG=E+F,H\n投票率\nH=C/G\n(%)
1,NaN,NaN,NaN,1\n張家豪\n台灣動物保護黨,2\n王文娟\n無,3\n鄭匡宇\n無,4\n黃聖峰\n台澎黨,5\n童文薰\n無,6\n蔣萬安\n中國國民黨,7\n蘇煥智\n台灣維新,...,11\n謝立康\n無,12\n陳時中\n民主進步黨,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,北投區,NaN,NaN,218,56,49,35,204,"51,433",177,...,20,"46,309","132,060","1,134","133,194",20,"133,214","64,212","197,426",67.47


## Importing the data of `縣表3-2-63000(臺北市)-111年臺北市縣市議員選舉候選人在各投開票所得票數一覽表.xls`

Check the list of worksheets before importing.

In [3]:
file_name = "data/縣表3-2-63000(臺北市)-111年臺北市縣市議員選舉候選人在各投開票所得票數一覽表.xls"
excel_file = pd.ExcelFile(file_name)
print(excel_file.sheet_names)

['第1選舉區', '第2選舉區', '第3選舉區', '第4選舉區', '第5選舉區', '第6選舉區', '第7選舉區', '第8選舉區']


## We can specify which sheet to import via `sheet_name` parameter

In [4]:
for sheetName in excel_file.sheet_names:
    df = pd.read_excel(file_name, sheet_name=sheetName)
    print(f"Shape of {sheetName}: {df.shape}")

Shape of 第1選舉區: (357, 36)
Shape of 第2選舉區: (250, 24)
Shape of 第3選舉區: (307, 27)
Shape of 第4選舉區: (243, 24)
Shape of 第5選舉區: (259, 29)
Shape of 第6選舉區: (375, 38)
Shape of 第7選舉區: (1771, 13)
Shape of 第8選舉區: (1771, 14)


## The `sheet_name` parameter also accepts integer as input

In [5]:
for i in range(len(excel_file.sheet_names)):
    df = pd.read_excel(file_name, sheet_name=i)
    print(f"Shape of the {i}th sheet: {df.shape}")

Shape of the 0th sheet: (357, 36)
Shape of the 1th sheet: (250, 24)
Shape of the 2th sheet: (307, 27)
Shape of the 3th sheet: (243, 24)
Shape of the 4th sheet: (259, 29)
Shape of the 5th sheet: (375, 38)
Shape of the 6th sheet: (1771, 13)
Shape of the 7th sheet: (1771, 14)


## Wrangling Data

## Once the import part is done, it is time to wrangle the messy spreadsheets into a tidy data format

> Tidy datasets are all alike, but every messy dataset is messy in its own way.
>
> [Hadley Wickham](http://hadley.nz/)

## What is tidy data?

> There are three interrelated rules which make a dataset tidy:
> 
> 1. Each variable must have its own column.
> 2. Each observation must have its own row.
> 3. Each value must have its own cell.

Source: <https://vita.had.co.nz/papers/tidy-data.pdf>

## Why tidy data?

Simply put, once our data is in tidy format, it is easy to use and quite convenient to transform into the designated format of visualization or modeling functions for the next stage of data science project.

## What makes our original workbooks messy?

- Combined cells
- Missing values
- Summations within observations
- Value in variable names

## Using two parameters to skip combined cells and transform thousands from `str` to `int` while importing workbook

- `skiprows=[0, 1, 3 ,4]`
- `thousands=","`

In [6]:
file_name = "data/縣表3-1-63000(臺北市)-111年臺北市縣市長選舉候選人在各投開票所得票數一覽表.xls"
df = pd.read_excel(file_name, skiprows=[0, 1, 3, 4], thousands=",")

In [7]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,1\n張家豪\n台灣動物保護黨,2\n王文娟\n無,3\n鄭匡宇\n無,4\n黃聖峰\n台澎黨,5\n童文薰\n無,6\n蔣萬安\n中國國民黨,7\n蘇煥智\n台灣維新,...,11\n謝立康\n無,12\n陳時中\n民主進步黨,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,北投區,NaN,NaN,218,56,49,35,204,51433,177,...,20,46309,132060,1134,133194,20,133214,64212,197426,67.47
1,NaN,建民里,1.0,3,1,0,0,7,324,1,...,0,329,844,7,851,0,851,322,1173,72.55
2,NaN,建民里,2.0,1,0,1,0,0,289,0,...,0,365,849,10,859,0,859,353,1212,70.87
3,NaN,建民里,3.0,2,0,0,0,0,297,0,...,1,329,865,7,872,0,872,325,1197,72.85
4,NaN,文林里,4.0,2,0,0,0,0,255,2,...,0,304,747,2,749,0,749,355,1104,67.84


## Updating `columns` attributes with new column names and drop columns

In [8]:
n_candidates = df.shape[1] - 11
columns_to_select = 3 + n_candidates
df = df.iloc[:, :columns_to_select]
df.rename(columns={"Unnamed: 0": "town", "Unnamed: 1": "village", "Unnamed: 2": "office"}, inplace=True)
df.head()

,town,village,office,1\n張家豪\n台灣動物保護黨,2\n王文娟\n無,3\n鄭匡宇\n無,4\n黃聖峰\n台澎黨,5\n童文薰\n無,6\n蔣萬安\n中國國民黨,7\n蘇煥智\n台灣維新,8\n黃珊珊\n無,9\n施奉先\n無,10\n唐新民\n共和黨,11\n謝立康\n無,12\n陳時中\n民主進步黨
0,北投區,NaN,NaN,218,56,49,35,204,51433,177,33492,33,34,20,46309
1,NaN,建民里,1.0,3,1,0,0,7,324,1,179,0,0,0,329
2,NaN,建民里,2.0,1,0,1,0,0,289,0,193,0,0,0,365
3,NaN,建民里,3.0,2,0,0,0,0,297,0,236,0,0,1,329
4,NaN,文林里,4.0,2,0,0,0,0,255,2,184,0,0,0,304


## Using `fillna()` to forward-fill `town` values

`ffill`: propagate last valid observation forward to next valid.

In [9]:
filled_towns = df["town"].fillna(method="ffill")
df = df.assign(town=filled_towns)
df.head()

,town,village,office,1\n張家豪\n台灣動物保護黨,2\n王文娟\n無,3\n鄭匡宇\n無,4\n黃聖峰\n台澎黨,5\n童文薰\n無,6\n蔣萬安\n中國國民黨,7\n蘇煥智\n台灣維新,8\n黃珊珊\n無,9\n施奉先\n無,10\n唐新民\n共和黨,11\n謝立康\n無,12\n陳時中\n民主進步黨
0,北投區,NaN,NaN,218,56,49,35,204,51433,177,33492,33,34,20,46309
1,北投區,建民里,1.0,3,1,0,0,7,324,1,179,0,0,0,329
2,北投區,建民里,2.0,1,0,1,0,0,289,0,193,0,0,0,365
3,北投區,建民里,3.0,2,0,0,0,0,297,0,236,0,0,1,329
4,北投區,文林里,4.0,2,0,0,0,0,255,2,184,0,0,0,304


## Removing subtotals and totals via `dropna()`

- To comply with the tidy data rule.
- To avoid wrong summations.

In [10]:
df = df.dropna()
df.head()

,town,village,office,1\n張家豪\n台灣動物保護黨,2\n王文娟\n無,3\n鄭匡宇\n無,4\n黃聖峰\n台澎黨,5\n童文薰\n無,6\n蔣萬安\n中國國民黨,7\n蘇煥智\n台灣維新,8\n黃珊珊\n無,9\n施奉先\n無,10\n唐新民\n共和黨,11\n謝立康\n無,12\n陳時中\n民主進步黨
1,北投區,建民里,1.0,3,1,0,0,7,324,1,179,0,0,0,329
2,北投區,建民里,2.0,1,0,1,0,0,289,0,193,0,0,0,365
3,北投區,建民里,3.0,2,0,0,0,0,297,0,236,0,0,1,329
4,北投區,文林里,4.0,2,0,0,0,0,255,2,184,0,0,0,304
5,北投區,文林里,5.0,2,1,1,0,0,280,1,215,0,1,0,356


## Transposing wide format into long format

In [11]:
df_long = pd.melt(df, id_vars=["town", "village", "office"],
                  var_name='candidate_info',
                  value_name='votes'
                 )
df_long.head()

,town,village,office,candidate_info,votes
0,北投區,建民里,1.0,1\n張家豪\n台灣動物保護黨,3
1,北投區,建民里,2.0,1\n張家豪\n台灣動物保護黨,1
2,北投區,建民里,3.0,1\n張家豪\n台灣動物保護黨,2
3,北投區,文林里,4.0,1\n張家豪\n台灣動物保護黨,2
4,北投區,文林里,5.0,1\n張家豪\n台灣動物保護黨,2


## Splitting candidate column

In [12]:
candidate_info_split = df_long["candidate_info"].str.split("\n", expand=True)
candidate_info_split

,0,1,2
0,1,張家豪,台灣動物保護黨
1,1,張家豪,台灣動物保護黨
2,1,張家豪,台灣動物保護黨
3,1,張家豪,台灣動物保護黨
4,1,張家豪,台灣動物保護黨
...,...,...,...
21055,12,陳時中,民主進步黨
21056,12,陳時中,民主進步黨
21057,12,陳時中,民主進步黨
21058,12,陳時中,民主進步黨


## Reassembling `DataFrame`

In [13]:
df_reassembled = pd.concat((df_long[["town", "village", "office"]], candidate_info_split, df_long[["votes"]]), axis=1)
df_reassembled.rename(columns={0: "number", 1: "candidate", 2: "party"}, inplace=True)

## Adjusting column data type

In [14]:
df_reassembled = df_reassembled.astype({"office": int, "number": int})
df_reassembled.dtypes

town         object
village      object
office        int64
number        int64
candidate    object
party        object
votes         int64
dtype: object

## Assembling codes

## Define a function called `melt_tidy_dataframe()` which assembles the previous cells

In [15]:
def melt_tidy_dataframe(df):
    # updating columns
    n_candidates = df.shape[1] - 11
    columns_to_select = 3 + n_candidates
    df = df.iloc[:, :columns_to_select]
    df.rename(columns={"Unnamed: 0": "town", "Unnamed: 1": "village", "Unnamed: 2": "office"}, inplace=True)
    # forward-fill town values
    filled_towns = df["town"].fillna(method="ffill")
    df = df.assign(town=filled_towns)
    # removing subtotals and totals
    df = df.dropna()
    # transposing
    df_long = pd.melt(df, id_vars=["town", "village", "office"],
                      var_name='candidate_info',
                      value_name='votes'
                     )
    # splitting candidate info
    candidate_info_split = df_long["candidate_info"].str.split("\n", expand=True)
    df_reassembled = pd.concat((df_long[["town", "village", "office"]], candidate_info_split, df_long[["votes"]]), axis=1)
    df_reassembled.rename(columns={0: "number", 1: "candidate", 2: "party"}, inplace=True)
    # adjusting data type
    df_reassembled = df_reassembled.astype({"office": int, "number": int, "votes": int})
    return df_reassembled

## Retrieving file names

In [16]:
list_dir = os.listdir("data")
file_names = [file for file in list_dir if ".xls" in file and "縣市" in file]
print(file_names)
print(len(file_names))

['縣表3-1-10005(苗栗縣)-111年苗栗縣縣市長選舉候選人在各投開票所得票數一覽表.xls', '縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls', '縣表3-2-64000(高雄市)-111年高雄市縣市議員選舉候選人在各投開票所得票數一覽表.xls', '縣表3-2-67000(臺南市)-111年臺南市縣市議員選舉候選人在各投開票所得票數一覽表.xls', '縣表3-1-10010(嘉義縣)-111年嘉義縣縣市長選舉候選人在各投開票所得票數一覽表.xls', '縣表3-2-10015(花蓮縣)-111年花蓮縣縣市議員選舉候選人在各投開票所得票數一覽表.xls', '縣表3-1-09020(金門縣)-111年金門縣縣市長選舉候選人在各投開票所得票數一覽表.xls', '縣表3-1-10004(新竹縣)-111年新竹縣縣市長選舉候選人在各投開票所得票數一覽表.xls', '縣表3-1-10016(澎湖縣)-111年澎湖縣縣市長選舉候選人在各投開票所得票數一覽表.xls', '縣表3-2-63000(臺北市)-111年臺北市縣市議員選舉候選人在各投開票所得票數一覽表.xls', '縣表3-2-10014(臺東縣)-111年臺東縣縣市議員選舉候選人在各投開票所得票數一覽表.xls', '縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls', '縣表3-1-65000(新北市)-111年新北市縣市長選舉候選人在各投開票所得票數一覽表.xls', '縣表3-2-10013(屏東縣)-111年屏東縣縣市議員選舉候選人在各投開票所得票數一覽表.xls', '縣表3-1-09007(連江縣)-111年連江縣縣市長選舉候選人在各投開票所得票數一覽表.xls', '縣表3-1-64000(高雄市)-111年高雄市縣市長選舉候選人在各投開票所得票數一覽表.xls', '縣表3-2-10010(嘉義縣)-111年嘉義縣縣市議員選舉候選人在各投開票所得票數一覽表.xls', '縣表3-1-10018(新竹市)-111年新竹市縣市長選舉候選人在各投開票所得票數一覽表.xls', '縣表3-2-10004(新竹縣)-111年新竹縣縣市議員選舉候選人在各投開票所得票數一覽表.xls', '

## Applying `melt_tidy_dataframe()` on all workbooks

In [17]:
for file_name in file_names:
    county = re.split("\(|\)", file_name.split("-")[2])[1]
    if "市長" in file_name:
        campaign = "縣市長"
    else:
        campaign = "縣市議員"
    print(f"{county}:{campaign}")

苗栗縣:縣市長
桃園市:縣市議員
高雄市:縣市議員
臺南市:縣市議員
嘉義縣:縣市長
花蓮縣:縣市議員
金門縣:縣市長
新竹縣:縣市長
澎湖縣:縣市長
臺北市:縣市議員
臺東縣:縣市議員
臺中市:縣市議員
新北市:縣市長
屏東縣:縣市議員
連江縣:縣市長
高雄市:縣市長
嘉義縣:縣市議員
新竹市:縣市長
新竹縣:縣市議員
屏東縣:縣市長
宜蘭縣:縣市長
南投縣:縣市議員
雲林縣:縣市長
嘉義市:縣市議員
臺南市:縣市長
彰化縣:縣市長
彰化縣:縣市議員
臺東縣:縣市長
苗栗縣:縣市議員
連江縣:縣市議員
澎湖縣:縣市議員
花蓮縣:縣市長
雲林縣:縣市議員
臺北市:縣市長
金門縣:縣市議員
基隆市:縣市議員
桃園市:縣市長
宜蘭縣:縣市議員
基隆市:縣市長
臺中市:縣市長
新竹市:縣市議員
新北市:縣市議員
南投縣:縣市長


In [18]:
concatenated_df = pd.DataFrame()
for file_name in file_names:
    county = re.split("\(|\)", file_name.split("-")[2])[1]
    if "市長" in file_name:
        campaign = "縣市長"
    else:
        campaign = "縣市議員"
    file_path = f"data/{file_name}"
    excel_file = pd.ExcelFile(file_path)
    sheet_names = excel_file.sheet_names
    for sheetName in sheet_names:
        if len(sheet_names) == 1:
            electoral_district = np.nan
        else:
            electoral_district = sheetName
        df = pd.read_excel(file_path, skiprows=[0, 1, 3, 4], thousands=",", sheet_name=sheetName)
        melted_tidy_dataframe = melt_tidy_dataframe(df)
        melted_tidy_dataframe["county"] = county
        melted_tidy_dataframe["campaign"] = campaign
        melted_tidy_dataframe["electoral_district"] = electoral_district
        concatenated_df = pd.concat((concatenated_df, melted_tidy_dataframe))
        print(f"Melting and tidying worksheet {sheetName} of {file_name}...")
concatenated_df = concatenated_df.reset_index(drop=True) # reset index for the concatenated dataframe

Melting and tidying worksheet 苗栗縣 of 縣表3-1-10005(苗栗縣)-111年苗栗縣縣市長選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第1選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第2選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第3選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第4選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第5選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第6選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第7選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第8選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第9選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第10選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一

Melting and tidying worksheet 第9選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第10選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第11選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第12選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第13選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第14選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第15選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第16選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第17選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 新北市 of 縣表3-1-65000(新北市)-111年新北市縣市長選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第1選舉區 of 縣表3-2-10013(屏東縣)-111年屏東縣縣市議員選舉候選人在各投

Melting and tidying worksheet 第5選舉區 of 縣表3-2-10016(澎湖縣)-111年澎湖縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第6選舉區 of 縣表3-2-10016(澎湖縣)-111年澎湖縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 花蓮縣 of 縣表3-1-10015(花蓮縣)-111年花蓮縣縣市長選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第1選舉區 of 縣表3-2-10009(雲林縣)-111年雲林縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第2選舉區 of 縣表3-2-10009(雲林縣)-111年雲林縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第3選舉區 of 縣表3-2-10009(雲林縣)-111年雲林縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第4選舉區 of 縣表3-2-10009(雲林縣)-111年雲林縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第5選舉區 of 縣表3-2-10009(雲林縣)-111年雲林縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第6選舉區 of 縣表3-2-10009(雲林縣)-111年雲林縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 臺北市 of 縣表3-1-63000(臺北市)-111年臺北市縣市長選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第1選舉區 of 縣表3-2-09020(金門縣)-111年金門縣縣市議員選舉候選人在各投開票所得票數一覽表.x

In [19]:
concatenated_df.head()

,town,village,office,number,candidate,party,votes,county,campaign,electoral_district
0,苗栗市,中苗里,1,1,鍾東錦,無,264,苗栗縣,縣市長,NaN
1,苗栗市,青苗里,2,1,鍾東錦,無,251,苗栗縣,縣市長,NaN
2,苗栗市,維祥里,3,1,鍾東錦,無,299,苗栗縣,縣市長,NaN
3,苗栗市,維祥里,4,1,鍾東錦,無,299,苗栗縣,縣市長,NaN
4,苗栗市,維祥里,5,1,鍾東錦,無,309,苗栗縣,縣市長,NaN


In [20]:
concatenated_df.tail()

,town,village,office,number,candidate,party,votes,county,campaign,electoral_district
409180,仁愛鄉,法治村,489,3,王永慶,無,6,南投縣,縣市長,NaN
409181,仁愛鄉,中正村,490,3,王永慶,無,6,南投縣,縣市長,NaN
409182,仁愛鄉,互助村,491,3,王永慶,無,2,南投縣,縣市長,NaN
409183,仁愛鄉,互助村,492,3,王永慶,無,3,南投縣,縣市長,NaN
409184,仁愛鄉,新生村,493,3,王永慶,無,4,南投縣,縣市長,NaN


## Define a class `TaiwanElection2022` to make the above codes more organized

```python
class TaiwanElection2022:
    def __init__(self, data_folder: None=None):
        list_dir = os.listdir(data_folder)
        self.data_folder = data_folder
        self.file_names = [file for file in list_dir if ".xls" in file and "縣市" in file]
    def melt_tidy_dataframe(self, df) -> pd.core.frame.DataFrame:
        # updating columns
        n_candidates = df.shape[1] - 11
        columns_to_select = 3 + n_candidates
        df = df.iloc[:, :columns_to_select]
        df.rename(columns={"Unnamed: 0": "town", "Unnamed: 1": "village", "Unnamed: 2": "office"}, inplace=True)
        # forward-fill town values
        filled_towns = df["town"].fillna(method="ffill")
        df = df.assign(town=filled_towns)
        # removing subtotals and totals
        df = df.dropna()
        # transposing
        df_long = pd.melt(df, id_vars=["town", "village", "office"],
                          var_name='candidate_info',
                          value_name='votes'
                         )
        # splitting candidate info
        candidate_info_split = df_long["candidate_info"].str.split("\n", expand=True)
        df_reassembled = pd.concat((df_long[["town", "village", "office"]], candidate_info_split, df_long[["votes"]]), axis=1)
        df_reassembled.rename(columns={0: "number", 1: "candidate", 2: "party"}, inplace=True)
        # adjusting data type
        df_reassembled = df_reassembled.astype({"office": int, "number": int, "votes": int})
        return df_reassembled
```

```python
    def create_election_dataframe(self) -> pd.core.frame.DataFrame:
        concatenated_df = pd.DataFrame()
        for file_name in self.file_names:
            county = re.split("\(|\)", file_name.split("-")[2])[1]
            if "市長" in file_name:
                campaign = "縣市長"
            else:
                campaign = "縣市議員"
            if self.data_folder is None:
                file_path = f"{file_name}"
            else:
                file_path = f"{self.data_folder}/{file_name}"
            excel_file = pd.ExcelFile(file_path)
            sheet_names = excel_file.sheet_names
            for sheetName in sheet_names:
                if len(sheet_names) == 1:
                    electoral_district = np.nan
                else:
                    electoral_district = sheetName
                df = pd.read_excel(file_path, skiprows=[0, 1, 3, 4], thousands=",", sheet_name=sheetName)
                melted_tidy_dataframe = self.melt_tidy_dataframe(df)
                melted_tidy_dataframe["county"] = county
                melted_tidy_dataframe["campaign"] = campaign
                melted_tidy_dataframe["electoral_district"] = electoral_district
                concatenated_df = pd.concat((concatenated_df, melted_tidy_dataframe))
                print(f"Melting and tidying worksheet {sheetName} of {file_name}...")
        concatenated_df = concatenated_df.reset_index(drop=True) # reset index for the concatenated dataframe
        out_df = concatenated_df[["county", "town", "village", "office", "number", "campaign", "electoral_district", "party", "candidate", "votes"]]
        return out_df
```

## Use `TaiwanElection2022` class to generate the compact `DataFrame`

Source: <https://raw.githubusercontent.com/datainpoint/classroom-programming-and-data-analysis/main/taiwan_election_2022.py>

In [21]:
from taiwan_election_2022 import TaiwanElection2022

tw_election_2022 = TaiwanElection2022("data")
election_dataframe = tw_election_2022.create_election_dataframe()

Melting and tidying worksheet 苗栗縣 of 縣表3-1-10005(苗栗縣)-111年苗栗縣縣市長選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第1選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第2選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第3選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第4選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第5選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第6選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第7選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第8選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第9選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第10選舉區 of 縣表3-2-68000(桃園市)-111年桃園市縣市議員選舉候選人在各投開票所得票數一

Melting and tidying worksheet 第9選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第10選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第11選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第12選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第13選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第14選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第15選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第16選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第17選舉區 of 縣表3-2-66000(臺中市)-111年臺中市縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 新北市 of 縣表3-1-65000(新北市)-111年新北市縣市長選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第1選舉區 of 縣表3-2-10013(屏東縣)-111年屏東縣縣市議員選舉候選人在各投

Melting and tidying worksheet 第5選舉區 of 縣表3-2-10016(澎湖縣)-111年澎湖縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第6選舉區 of 縣表3-2-10016(澎湖縣)-111年澎湖縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 花蓮縣 of 縣表3-1-10015(花蓮縣)-111年花蓮縣縣市長選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第1選舉區 of 縣表3-2-10009(雲林縣)-111年雲林縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第2選舉區 of 縣表3-2-10009(雲林縣)-111年雲林縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第3選舉區 of 縣表3-2-10009(雲林縣)-111年雲林縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第4選舉區 of 縣表3-2-10009(雲林縣)-111年雲林縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第5選舉區 of 縣表3-2-10009(雲林縣)-111年雲林縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第6選舉區 of 縣表3-2-10009(雲林縣)-111年雲林縣縣市議員選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 臺北市 of 縣表3-1-63000(臺北市)-111年臺北市縣市長選舉候選人在各投開票所得票數一覽表.xls...
Melting and tidying worksheet 第1選舉區 of 縣表3-2-09020(金門縣)-111年金門縣縣市議員選舉候選人在各投開票所得票數一覽表.x

In [22]:
election_dataframe

,county,town,village,office,number,campaign,electoral_district,party,candidate,votes
0,苗栗縣,苗栗市,中苗里,1,1,縣市長,NaN,無,鍾東錦,264
1,苗栗縣,苗栗市,青苗里,2,1,縣市長,NaN,無,鍾東錦,251
2,苗栗縣,苗栗市,維祥里,3,1,縣市長,NaN,無,鍾東錦,299
3,苗栗縣,苗栗市,維祥里,4,1,縣市長,NaN,無,鍾東錦,299
4,苗栗縣,苗栗市,維祥里,5,1,縣市長,NaN,無,鍾東錦,309
...,...,...,...,...,...,...,...,...,...,...
409180,南投縣,仁愛鄉,法治村,489,3,縣市長,NaN,無,王永慶,6
409181,南投縣,仁愛鄉,中正村,490,3,縣市長,NaN,無,王永慶,6
409182,南投縣,仁愛鄉,互助村,491,3,縣市長,NaN,無,王永慶,2
409183,南投縣,仁愛鄉,互助村,492,3,縣市長,NaN,無,王永慶,3


## Done wrangling data of 2022 Taiwan election!

![](https://media.giphy.com/media/1sjwSoZLcENCE/giphy.gif)

Source: <https://giphy.com/>